In [20]:
import networkx as nx
import matplotlib.pyplot as plt

from sp_rest_client import *

class SpGraph:
    def __init__(self):
        self.nx_graph = nx.Graph()
        self.person_dict = {}
        self.company_dict = {}
        self.company_person_dict = {}

    def add_person(self, person):
        person_key = self.__get_person_key(person['id'])
        
        if person_key in self.person_dict:
            return False
        
        self.nx_graph.add_node(person_key)
        self.person_dict[person_key] = person
        return True;
        
    def add_company(self, company):
        company_key = self.__get_company_key(company['id'])
        
        if company_key in self.company_dict:
            return False
        
        self.nx_graph.add_node(company_key)
        self.person_dict[company_key] = company
        return True
        
    def add_company_person(self, person_id, company_id, company_person):
        person_key = self.__get_person_key(person_id)
        company_key = self.__get_company_key(company_id)
        company_person_key = company_key + person_key
        
        if company_person_key in self.company_person_dict:
            return False
        
        self.nx_graph.add_edge(company_key, person_key)
        self.company_person_dict[company_person_key] = company_person
        return True
    
    def __get_person_key(self, person_id):
        return 'p_' + person_id
    
    def __get_company_key(self, company_id):
        return 'c_' + company_id
        
    def draw():
        nx.draw_networkx(self.G)
        plt.show()
    
def process_person(id, slug, d, max_d):
    person = client.person(id, slug)
    person_info = person['information']
    
    print('-' * (d * 2) + person_info['name'] + ', ' + person_info['birthYear'] + ', ' + person_info['id'])
    if graph.add_person(person_info):
        for company_person in person['companies']:
            if d < max_d:
                process_company(company_person['id'], company_person['slug'], d, max_d)
                
            graph.add_company_person(company_person['id'], id, company_person)
        
def process_company(id, slug, d, max_d):
    company = client.company(id, slug)
    company_info = company['information']
    
    print('-' * (d * 2 + 1) + company_info['name'] + ', ' + company_info['krs'] + ', ' + company_info['id'])
    if graph.add_company(company_info):
        for representation_person in company['representation']:
            process_person(representation_person['id'], representation_person['slug'], d + 1, max_d)
        for directors_board_person in company['directorsBoard']:
            process_person(directors_board_person['id'], directors_board_person['slug'], d + 1, max_d)

graph = SpGraph()
client = SpRestClient()

print('START!')
results = client.search('Jan Kowalski')
results_p = [result for result in results if result['type'] == 'person']

for result_p in results_p:
    process_person(result_p['id'], result_p['slug'], 0, 3)

graph.draw()

START!
Jan Kowalski, 1934
-ZWIĄZEK ZAWODOWY PRACOWNIKÓW SŁUŻBY ZDROWIA PRZY S.P. ZESPOLE OPIEKI ZDROWOTNEJ W BRZESKU, 0000033593
--Zdzisław Curyło, 1933
---ZWIĄZEK ZAWODOWY PRACOWNIKÓW SŁUŻBY ZDROWIA PRZY S.P. ZESPOLE OPIEKI ZDROWOTNEJ W BRZESKU, 0000033593
----Zdzisław Curyło, 1933
----Jan Kowalski, 1934
----Zofia Majcher, 1966
-----ZWIĄZEK ZAWODOWY PRACOWNIKÓW SŁUŻBY ZDROWIA PRZY S.P. ZESPOLE OPIEKI ZDROWOTNEJ W BRZESKU, 0000033593
------Zdzisław Curyło, 1933
------Jan Kowalski, 1934
------Zofia Majcher, 1966
------Jadwiga Górska, 1958
------Mieczysław Łakoma, 1947
------Zofia Chmielarz, 1940
------Jadwiga Marzec Damasiewicz, 1958
------Irena Mikołajek, 1960
----Jadwiga Górska, 1958
----Mieczysław Łakoma, 1947
----Zofia Chmielarz, 1940
----Jadwiga Marzec Damasiewicz, 1958
----Irena Mikołajek, 1960
--Jan Kowalski, 1934
--Zofia Majcher, 1966
--Jadwiga Górska, 1958
--Mieczysław Łakoma, 1947
--Zofia Chmielarz, 1940
--Jadwiga Marzec Damasiewicz, 1958
--Irena Mikołajek, 1960
Jan Kowalski, 

TypeError: a bytes-like object is required, not 'NoneType'